# Deep Learning
### Assignment 2

Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

data as a flat matrix,
labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

- First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

        with graph.as_default():
            ...
    
- Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

        with tf.Session(graph=graph) as session:
            ...
    
Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [6]:
train_subset = 10000

#define a tesnorflow graph
graph = tf.Graph()
with graph.as_default():
    #Constants
    tf_train_dataset  = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits = logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights) + biases)
    

Let's run this computation and iterate:

In [9]:
print(train_labels[:5])

[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [14]:
num_steps = 801

def accuracy(prediction, labels):
    return (100.0 * np.sum(np.argmax(prediction,1)==np.argmax(labels,1))/ prediction.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _,l_,predictions = session.run([optimizer,loss,train_prediction])
        if (step%100 == 0):
            print('Loss at step %d: %f' %(step, l_))
            print('Training accuracy is : %.1f%%' % accuracy(predictions,train_labels[:train_subset]))
            print('Validation accuracy is : %.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy is : %.1f%%' % accuracy(test_prediction.eval(),test_labels))

Initialized
Loss at step 0: 17.311646
Training accuracy is : 11.8%
Validation accuracy is : 13.7%
Loss at step 100: 2.324468
Training accuracy is : 71.9%
Validation accuracy is : 70.6%
Loss at step 200: 1.866498
Training accuracy is : 74.8%
Validation accuracy is : 72.6%
Loss at step 300: 1.621826
Training accuracy is : 76.3%
Validation accuracy is : 73.3%
Loss at step 400: 1.458429
Training accuracy is : 77.0%
Validation accuracy is : 73.7%
Loss at step 500: 1.336828
Training accuracy is : 77.7%
Validation accuracy is : 74.0%
Loss at step 600: 1.240813
Training accuracy is : 78.5%
Validation accuracy is : 74.1%
Loss at step 700: 1.162193
Training accuracy is : 79.0%
Validation accuracy is : 74.2%
Loss at step 800: 1.096270
Training accuracy is : 79.5%
Validation accuracy is : 74.5%
Test accuracy is : 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [1]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Parameters
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    logits = tf.matmul(tf_train_dataset,weights)+biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels,logits= logits))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


NameError: name 'tf' is not defined

Let's run it :

In [28]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_label = train_labels[offset:(offset+batch_size)]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_label}
        _,l,prediction = session.run([optimizer,loss,train_prediction],feed_dict = feed_dict)
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(),valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.978466
Minibatch accuracy: 0.0%


C:\Users\ashis\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Validation accuracy: 11.6%
Minibatch loss at step 500: 1.304307
Minibatch accuracy: 0.0%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.560361
Minibatch accuracy: 0.0%
Validation accuracy: 76.5%
Minibatch loss at step 1500: 0.731815
Minibatch accuracy: 0.0%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 0.963666
Minibatch accuracy: 0.0%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.007297
Minibatch accuracy: 0.0%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.986212
Minibatch accuracy: 0.0%
Validation accuracy: 79.2%
Test accuracy: 86.7%


---


## Problem

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.


---

In [29]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Parameters
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, 1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    
    weights_2 = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    tf_relu_result = tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    logits = tf.matmul(tf_relu_result,weights_2)+biases_2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels,logits= logits))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
        tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(
        tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2)+biases_2)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as sess:
    print("Initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step*batch_size)%(train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels: batch_labels}
        _,l,prediction = sess.run([optimizer,loss,train_prediction],feed_dict = feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_label))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(),valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 317.195557
Minibatch accuracy: 0.0%


C:\Users\ashis\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Validation accuracy: 29.2%
Minibatch loss at step 500: 27.467537
Minibatch accuracy: 0.0%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 8.644811
Minibatch accuracy: 0.0%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 7.072350
Minibatch accuracy: 0.0%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 3.777250
Minibatch accuracy: 0.0%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 8.091771
Minibatch accuracy: 0.0%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 2.144358
Minibatch accuracy: 0.0%
Validation accuracy: 82.2%
Test accuracy: 89.4%
